In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!sudo apt-get install -y tesseract-ocr poppler-utils
!pip install -q transformers gradio torch accelerate Pillow PyPDF2 pdf2image pytesseract datasets "diffusers[torch]" moviepy opencv-python-headless


Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 1s (7,758 kB/s)
debconf: unable to

In [ ]:
import json

dataset_path = "/content/drive/MyDrive/annotated_dataset_all.json"  # Adjust path if needed
with open(dataset_path, "r") as f:
    annotated_data = json.load(f)

print(f"Loaded {len(annotated_data)} examples.")


Loaded 434 examples.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `personal` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `personal`

In [ ]:

import os
import torch
from datasets import Dataset
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, TrainingArguments, Trainer
from PIL import Image

# Load the processor and model (remove torch_dtype=torch.float16)
processor = AutoProcessor.from_pretrained("google/paligemma-3b-mix-224")
model = PaliGemmaForConditionalGeneration.from_pretrained(
    "google/paligemma-3b-mix-224",
    device_map="auto"  # Model loaded in default dtype (FP32)
)

def preprocess_example(example):
    outputs = []
    image_file = example.get("image_path", "")
    image_files = [image_file] if isinstance(image_file, str) else image_file

    for path in image_files:
        if not os.path.exists(path):
            raise ValueError(f"Image file not found: {path}")
        image = Image.open(path).convert("RGB")
        input_text = "<image> " + example["ocr_text"]
        target_text = "<image> " + example["annotation_text"]


        inputs = processor(
            text=input_text,
            images=[image],
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=768
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}  # Remove batch dimension


        target = processor(
            text=target_text,
            images=[image],
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=768
        ).input_ids.squeeze(0)  # Remove batch dimension

        inputs["labels"] = target
        outputs.append(inputs)
    return outputs


processed_examples = []
for example in annotated_data:
    processed_examples.extend(preprocess_example(example))

hf_dataset = Dataset.from_list(processed_examples)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Assuming tokenized_dataset is defined from your previous cell.
print(hf_dataset)
example = hf_dataset[0]

print("input_ids length:", len(example["input_ids"]))
print("First 10 input_ids:", example["input_ids"][:10])

print("attention_mask length:", len(example["attention_mask"]))
print("First 10 attention_mask values:", example["attention_mask"][:10])

print("pixel_values shape:", len(example["pixel_values"]), "x", len(example["pixel_values"][0]))
print("labels length:", len(example["labels"]))
print("First 10 label token IDs:", example["labels"][:10])




Dataset({
    features: ['input_ids', 'attention_mask', 'pixel_values', 'labels'],
    num_rows: 434
})
input_ids length: 1024
First 10 input_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask length: 1024
First 10 attention_mask values: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
pixel_values shape: 3 x 224
labels length: 1024
First 10 label token IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def custom_data_collator(features):
    batch = {}
    for key in features[0]:
        values = [torch.tensor(feature[key]) if not isinstance(feature[key], torch.Tensor) else feature[key] for feature in features]
        batch[key] = torch.stack(values)
    return batch

# Training arguments with fp16 enabled
training_args = TrainingArguments(
    output_dir="./paligemma_finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=200,
    fp16=True,  # Keep fp16 enabled for mixed precision training
    report_to=[],
    eval_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset,
    data_collator=custom_data_collator
)

trainer.train()

model.save_pretrained("./paligemma_finetuned")
processor.save_pretrained("./paligemma_finetuned")

`labels` contains `pad_token_id` which will be masked with `config.ignore_index`. You have to mask out `pad_token_id` when preparing `labels`, this behavior will be removed in v.4.46.
/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/configuration_paligemma.py:135: FutureWarning: The `ignore_index` attribute is deprecated and will be removed in v4.47.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.10 GiB. GPU 0 has a total capacity of 39.56 GiB of which 346.88 MiB is free. Process 55460 has 39.21 GiB memory in use. Of the allocated memory 37.90 GiB is allocated by PyTorch, and 816.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import gradio as gr
import tempfile
import moviepy.editor as mpy
from diffusers import StableDiffusionPipeline

sd_pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
sd_pipeline = sd_pipeline.to("cuda")

def answer_question(image, question):
    """
    Uses the fine-tuned Paligemma vision-language model to answer the question.
    Also generates an illustrative image using Stable Diffusion and a video slideshow from the image.
    """

    prompt = f"Explain step-by-step using NCERT Class 10 concepts: {question}"


    if image is not None:
        image = image.convert("RGB")

    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to("cuda")


    outputs = model.generate(**inputs, max_new_tokens=500)
    answer_text = processor.decode(outputs[0], skip_special_tokens=True)


    image_prompt = f"Illustration of: {question}"
    gen_image = sd_pipeline(image_prompt).images[0]


    frames = [gen_image for _ in range(5)]
    video_path = tempfile.mktemp(suffix=".mp4")
    clip = mpy.ImageSequenceClip(frames, fps=1)
    clip.write_videofile(video_path, codec="libx264", audio=False)

    return answer_text, gen_image, video_path

interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Image(label="Upload an Image (optional)", type="pil"),
        gr.Textbox(label="Enter your question", placeholder="Type your question here...")
    ],
    outputs=[
        gr.Textbox(label="Answer (Text)"),
        gr.Image(label="Answer (Image)"),
        gr.Video(label="Answer (Video)")
    ],
    title="NCERT Virtual Tutor",
    description="Ask a question and optionally upload an image to receive an explanation in text, an illustrative image, and a short video."
)

interface.launch(share=True)


In [ ]:
#deepseekwalaresponse

In [ ]:
import os
import torch
from datasets import Dataset
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, TrainingArguments, Trainer
from PIL import Image


processor = AutoProcessor.from_pretrained("google/paligemma-3b-mix-224")
model = PaliGemmaForConditionalGeneration.from_pretrained(
    "google/paligemma-3b-mix-224",
    device_map="auto",
    torch_dtype=torch.bfloat16  # Use BF16 for better stability & lower memory
)

def preprocess_example(example):
    outputs = []
    image_file = example.get("image_path", "")
    image_files = [image_file] if isinstance(image_file, str) else image_file

    for path in image_files:
        if not os.path.exists(path):
            raise ValueError(f"Image file not found: {path}")
        image = Image.open(path).convert("RGB").resize((224, 224))
        input_text = "<image> " + example["ocr_text"]
        target_text = "<image> " + example["annotation_text"]


        inputs = processor(
            text=input_text,
            images=[image],
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=768
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}  # Remove batch dimension


        target = processor(
            text=target_text,
            images=[image],
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=768
        ).input_ids.squeeze(0)  # Remove batch dimension

        inputs["labels"] = target
        outputs.append(inputs)
    return outputs

processed_examples = []
for example in annotated_data:
    processed_examples.extend(preprocess_example(example))

hf_dataset = Dataset.from_list(processed_examples)

In [ ]:
def custom_data_collator(features):
    batch = {}
    for key in features[0]:
        values = [torch.tensor(feature[key]) if not isinstance(feature[key], torch.Tensor) else feature[key] for feature in features]
        batch[key] = torch.stack(values)
    return batch


training_args = TrainingArguments(
    output_dir="./paligemma_finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    num_train_epochs=10,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=200,
    bf16=True,  # Keep fp16 enabled for mixed precision training
    report_to=[],
    eval_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset,
    data_collator=custom_data_collator
)

trainer.train()

model.save_pretrained("./paligemma_finetuned")
processor.save_pretrained("./paligemma_finetuned")

`labels` contains `pad_token_id` which will be masked with `config.ignore_index`. You have to mask out `pad_token_id` when preparing `labels`, this behavior will be removed in v.4.46.
/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/configuration_paligemma.py:135: FutureWarning: The `ignore_index` attribute is deprecated and will be removed in v4.47.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,45.688800
20,41.028700
30,39.649100
40,38.599700
50,37.227200
60,36.155900
70,36.772500
80,35.642800
90,35.173300
100,34.366100


/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/configuration_paligemma.py:135: FutureWarning: The `ignore_index` attribute is deprecated and will be removed in v4.47.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/configuration_paligemma.py:135: FutureWarning: The `ignore_index` attribute is deprecated and will be removed in v4.47.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/configuration_paligemma.py:135: FutureWarning: The `ignore_index` attribute is deprecated and will be removed in v4.47.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/configuration_paligemma.py:135: FutureWarning: The `ignore_index` attribute is deprecated and will be removed in v4.47.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/paligemma/configuration_paligemma.py:135: FutureWarning: The `ignore_index` attribute is deprecated and will be 

[]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import os
from google.colab import files

folder_path = "./paligemma_finetuned"  # or any folder you want to download files from
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    # This will prompt a download for each file.
    files.download(file_path)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Copy the fine-tuned model folder to your Google Drive.
!cp -r ./paligemma_finetuned /content/drive/MyDrive/
